drive bağlantısı

In [ ]:
# === Hücre 1: Ortamı düzelt (NumPy binary mismatch fix) ===
%pip -q install --upgrade pip
# Eski/broken NumPy'i temizle, uyumlu (py3.12) wheel'i zorla kur
%pip -q uninstall -y numpy
%pip -q install --no-cache-dir --force-reinstall "numpy==2.1.1"

# Ultralytics'i da güncelle (senin mesajındaki 8.3.196)
%pip -q install --no-cache-dir --upgrade "ultralytics==8.3.196" "opencv-python-headless<4.11" "pillow<11"



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.1 which is incompatible.


In [ ]:
%pip -q install --upgrade ultralytics==8.2.0 pillow opencv-python tqdm

from google.colab import drive
drive.mount('/content/drive')

# Kaynaklar (Drive'daki yapıya göre)
VIOL_ROOT = '/content/drive/MyDrive/censorly-dataset/Violence'
DST_ROOT  = '/content/datasets/violence'                         # YOLO'ya dönüştürülmüş birleşik veri
RUN_DIR   = '/content/drive/MyDrive/censorly-training/violence'  # eğitim çıktılarını Drive'a yaz

import os, shutil, glob, random
from pathlib import Path
os.makedirs(DST_ROOT, exist_ok=True)
os.makedirs(RUN_DIR, exist_ok=True)
print('OK')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 24.3 MB/s eta 0:00:00
Mounted at /content/drive
OK


In [ ]:
# === Hücre 2: Eğitim (A100, YOLOv8m, 70 epoch) ===
from pathlib import Path
import yaml, datetime
from ultralytics import YOLO

# Yollar
DRIVE_ROOT = "/content/drive/MyDrive"
DATA_ROOT  = Path(f"{DRIVE_ROOT}/censorly-dataset/Violence/Violence-True/Violence")
RUN_ROOT   = Path(f"{DRIVE_ROOT}/censorly-training/violence")
images_dir = DATA_ROOT / "images"

# Split bul
def pick_split(cands):
    for n in cands:
        p = images_dir / n
        if p.exists(): return n
    return None

train_split = pick_split(["train","Train","TRAIN"])
val_split   = pick_split(["validation","val","Validation","VAL"])
test_split  = pick_split(["test","Test","TEST"])
assert train_split, "train split bulunamadı"
assert val_split,   "validation/val split bulunamadı"

train_path = str(images_dir / train_split)
val_path   = str(images_dir / val_split)
test_path  = str(images_dir / test_split) if test_split else None

# data.yaml
RUN_ROOT.mkdir(parents=True, exist_ok=True)
yaml_path = RUN_ROOT / "data_violence.yaml"
data_yaml = {
    "path": str(DATA_ROOT),
    "train": train_path,
    "val":   val_path,
    **({"test": test_path} if test_path else {}),
    "nc": 1,
    "names": ["violence"],
}
with open(yaml_path, "w") as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False)
print(f"[INFO] data.yaml -> {yaml_path}")

# Eğitim
MODEL  = "yolov8m.pt"
IMG    = 960
EPOCHS = 70
run_stamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
exp_name  = f"yolov8m_960_e70_{run_stamp}"

model = YOLO(MODEL)
results = model.train(
    data=str(yaml_path),
    epochs=EPOCHS,
    imgsz=IMG,
    device=0,          # A100
    batch=-1,
    workers=8,
    project=str(RUN_ROOT),
    name=exp_name,
    exist_ok=True,
    seed=42,
    cache=True,
    patience=20,
    save=True,
    save_period=0,
    pretrained=True,
    verbose=True,
)

print("\n[OK] Eğitim başladı/bitti loglara bakabilirsin.")
print(f"Çıktı klasörü: {RUN_ROOT / exp_name}")
print("Best/last:     best.pt, last.pt")
print(f"TensorBoard:   !tensorboard --logdir {RUN_ROOT}")

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
from pathlib import Path
import shutil

# Çıktı klasöründeki best.pt
src = Path("/content/drive/MyDrive/censorly-training/violence/yolov8m_960_e70_20250908_2054/weights/best.pt")

# Kaydedilmesini istediğin hedef
dst_dir = Path("/content/drive/MyDrive/censorly-models/violence")
dst_dir.mkdir(parents=True, exist_ok=True)
dst = dst_dir / "violence_best.pt"

shutil.copy(src, dst)
print(f"[OK] best.pt kopyalandı -> {dst}")

[OK] best.pt kopyalandı -> /content/drive/MyDrive/censorly-models/violence/violence_best.pt


In [ ]:
import shutil
from pathlib import Path

DRIVE_ROOT = "/content/drive/MyDrive"
FALSE_ROOT = Path(f"{DRIVE_ROOT}/censorly-dataset/Violence/Violence-False")

# Kaynak klasörler
SRC_FOLDERS = [
    FALSE_ROOT / "Scissors",
    FALSE_ROOT / "Kitchen knife",
    FALSE_ROOT / "Hammer"
]

# Hedef klasörler
DST_IMAGES = FALSE_ROOT / "images"
DST_LABELS = FALSE_ROOT / "labels"

splits = ["train", "validation", "val", "test"]

for split in splits:
    (DST_IMAGES / split).mkdir(parents=True, exist_ok=True)
    (DST_LABELS / split).mkdir(parents=True, exist_ok=True)

copied, labels_created = 0, 0

for src in SRC_FOLDERS:
    for split in splits:
        src_images = src / "images" / split
        src_labels = src / "labels" / split
        if not src_images.exists():
            continue

        # Görselleri kopyala
        for img in src_images.glob("*.*"):
            dst_img = DST_IMAGES / split / img.name
            shutil.copy(img, dst_img)
            copied += 1

            # Boş label dosyası oluştur
            dst_lbl = DST_LABELS / split / f"{img.stem}.txt"
            dst_lbl.write_text("", encoding="utf-8")
            labels_created += 1

print(f"[OK] {copied} görsel kopyalandı, {labels_created} boş label oluşturuldu.")
print(f"Hedef klasör -> {FALSE_ROOT}")

[OK] 594 görsel kopyalandı, 594 boş label oluşturuldu.
Hedef klasör -> /content/drive/MyDrive/censorly-dataset/Violence/Violence-False


In [ ]:
from pathlib import Path

DRIVE = "/content/drive/MyDrive"
TRUE  = Path(f"{DRIVE}/censorly-dataset/Violence/Violence-True/Violence")
FALSE = Path(f"{DRIVE}/censorly-dataset/Violence/Violence-False")
RUN   = Path(f"{DRIVE}/censorly-training/violence")
RUN.mkdir(parents=True, exist_ok=True)

def count_imgs(p):
    exts = (".jpg",".jpeg",".png",".bmp",".webp",".JPG",".PNG",".JPEG",".WEBP")
    return sum(1 for x in p.rglob("*") if x.suffix in exts)

splits = ["train","validation","test"]
for side, root in [("TRUE", TRUE), ("FALSE", FALSE)]:
    print(f"\n[{side}]")
    for s in splits:
        d = root / "images" / s
        print(f"  {s:<11}", d, "->", count_imgs(d) if d.exists() else 0)


[TRUE]
  train       /content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence/images/train -> 2536
  validation  /content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence/images/validation -> 289
  test        /content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence/images/test -> 151

[FALSE]
  train       /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/train -> 250
  validation  /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/validation -> 99
  test        /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/test -> 63


In [ ]:
from pathlib import Path
import os

DRIVE = "/content/drive/MyDrive"
FALSE_ROOT = Path(f"{DRIVE}/censorly-dataset/Violence/Violence-False")
IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff",".JPG",".PNG",".JPEG",".WEBP",".TIF",".TIFF"}

created, emptied = 0, 0

def ensure_empty_labels(split):
    global created, emptied
    img_dir = FALSE_ROOT / "images" / split
    lbl_dir = FALSE_ROOT / "labels" / split
    if not img_dir.exists():
        return
    lbl_dir.mkdir(parents=True, exist_ok=True)

    img_stems = {p.stem for p in img_dir.rglob("*") if p.is_file() and p.suffix in IMG_EXTS}
    lbl_paths = {p.stem: p for p in lbl_dir.rglob("*.txt") if p.is_file()}

    # Eksik: boş txt oluştur
    for stem in sorted(img_stems - set(lbl_paths.keys())):
        (lbl_dir / f"{stem}.txt").write_text("")
        created += 1

    # Var olanları boşalt (içi doluysa)
    for stem, lp in lbl_paths.items():
        if stem in img_stems and lp.stat().st_size > 0:
            lp.write_text("")
            emptied += 1

# Validation ve Test için uygula
for split in ["validation", "test"]:
    ensure_empty_labels(split)

# Cache temizliği
cache_removed = 0
for p in Path(f"{DRIVE}/censorly-dataset/Violence").rglob("*.cache"):
    try:
        os.remove(p)
        cache_removed += 1
    except Exception:
        pass

print(f"[FIX] Oluşturulan boş label: {created}")
print(f"[FIX] Boşaltılan (önceden dolu) label: {emptied}")
print(f"[FIX] Silinen .cache dosyası: {cache_removed}")

[FIX] Oluşturulan boş label: 154
[FIX] Boşaltılan (önceden dolu) label: 0
[FIX] Silinen .cache dosyası: 4


In [ ]:
from pathlib import Path
import datetime, json

DRIVE = "/content/drive/MyDrive"
RUN   = Path(f"{DRIVE}/censorly-training/violence")
RUN.mkdir(parents=True, exist_ok=True)

TRUE  = f"{DRIVE}/censorly-dataset/Violence/Violence-True/Violence"
FALSE = f"{DRIVE}/censorly-dataset/Violence/Violence-False"

def keep_existing(xs):
    return [x for x in xs if Path(x).exists()]

train_dirs = keep_existing([f"{TRUE}/images/train", f"{FALSE}/images/train"])
val_dirs   = keep_existing([
    f"{TRUE}/images/validation", f"{TRUE}/images/test",
    f"{FALSE}/images/validation", f"{FALSE}/images/test"
])
test_dirs  = keep_existing([f"{TRUE}/images/test", f"{FALSE}/images/test"])

yaml_text = f"""# auto-generated (YOLOv11)
path: .
train: {json.dumps(train_dirs)}
val:   {json.dumps(val_dirs if val_dirs else train_dirs)}
test:  {json.dumps(test_dirs if test_dirs else val_dirs)}
names: [violence]
nc: 1
"""

stamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
yaml_path = RUN / f"data_violence_v11_{stamp}.yaml"
yaml_path.write_text(yaml_text)
print("[INFO] data.yaml ->", yaml_path, "\n", yaml_text)

[INFO] data.yaml -> /content/drive/MyDrive/censorly-training/violence/data_violence_v11_20250918_1454.yaml 
 # auto-generated (YOLOv11)
path: .
train: ["/content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence/images/train", "/content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/train"]
val:   ["/content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence/images/validation", "/content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence/images/test", "/content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/validation", "/content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/test"]
test:  ["/content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence/images/test", "/content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/test"]
names: [violence]
nc: 1



In [ ]:

from ultralytics import YOLO
from pathlib import Path
import datetime, torch, os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("GPU:", torch.cuda.get_device_name(0))

RUN   = Path("/content/drive/MyDrive/censorly-training/violence")
DATA  = str(sorted(RUN.glob("data_violence_v11_*.yaml"))[-1])  # son yaml
MODEL = "yolo11m.pt"   # denge için m (n/s/m/l/x seçenekleri)
EPOCHS= 80
IMGSZ = 960

stamp  = datetime.datetime.now().strftime("%Y%m%d_%H%M")
RUN_NM = f"yolo11m_violence_{IMGSZ}_e{EPOCHS}_{stamp}"

model = YOLO(MODEL)
res = model.train(
    data=DATA,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=-1,              # AutoBatch (A100 ile rahat)
    project=str(RUN),
    name=RUN_NM,
    device=0,
    workers=8,
    deterministic=True,
    cache="disk",
    seed=42,

    # --- Geometri (dik + açılı) ---
    degrees=20.0,
    shear=5.0,
    perspective=0.0008,
    translate=0.08,
    scale=0.35,

    # --- Renk / kontrast (B/W + düşük ışık) ---
    hsv_h=0.015,
    hsv_s=0.35,
    hsv_v=0.35,

    # --- Blur & grayscale (hedefli) ---
    # (Ultralytics default Albumentations: Blur/MedianBlur/ToGray/CLAHE zaten etkin;
    #  burada olasılıkları default’a bırakıyoruz; aşırıya kaçmıyoruz.)

    # --- Kompozitler (formu bozmayacak seviyede) ---
    mosaic=0.35,
    mixup=0.0,
    copy_paste=0.0,
    fliplr=0.5, flipud=0.0,

    patience=100,          # early stop
    verbose=True
)

print("\n[OK] Train run:", RUN/ RUN_NM)
print("Weights:", RUN/ RUN_NM / "weights")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
GPU: NVIDIA A100-SXM4-40GB
New https://pypi.org/project/ultralytics/8.3.202 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.200 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/censorly-training/violence/data_violence_v11_20250918_1454.yaml, degrees=20.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, 

In [ ]:
# Eski iyi run'ından best.pt'yi göster (örnek: yolov8m_ft_rotgray_fix/best.pt)
from pathlib import Path

BASE_BEST = Path(f"{DRIVE}/censorly-training/violence/yolov8m_ft_rotgray_fix/weights/best.pt")
assert BASE_BEST.exists(), f"Bulunamadı: {BASE_BEST}"
print("[INFO] Fine-tune base:", BASE_BEST)

[INFO] Fine-tune base: /content/drive/MyDrive/censorly-training/violence/yolov8m_ft_rotgray_fix/weights/best.pt


In [ ]:
from ultralytics import YOLO
import numpy as np, json, os
m = YOLO("/content/drive/MyDrive/censorly-training/violence/yolov8m_ft_neg_960_e40_20250918_1604/weights/best.pt")
best = {"conf":0.25,"f1":-1}
for c in np.linspace(0.15, 0.60, 10):
    r = m.val(data="/content/drive/MyDrive/censorly-training/violence/data_violence_v11_ft_20250918_1604.yaml",
              conf=float(c), iou=0.60, imgsz=960, device=0, verbose=False)
    if r.results_dict["metrics/precision(B)"] + r.results_dict["metrics/recall(B)"]>0:
        f1 = 2*r.results_dict["metrics/precision(B)"]*r.results_dict["metrics/recall(B)"]/(
             r.results_dict["metrics/precision(B)"]+r.results_dict["metrics/recall(B)"])
        if f1>best["f1"]: best={"conf":float(c),"f1":float(f1)}
print(best)

Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.1 ms, read: 17.4±15.4 MB/s, size: 488.1 KB)
val: Scanning /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/labels/test... 602 images, 162 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 602/602 27.7it/s 21.7s
val: New cache created: /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 38/38 4.8it/s 7.9s
                   all        602        645      0.729      0.636      0.723      0.494
Speed: 0.7ms preprocess, 3.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /content/runs/detect/val3
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access 

In [ ]:
model = YOLO(str(BASE_BEST))  # önceki FT modelinden DEVAM

res = model.train(
    data=str(yaml_path),
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=-1,                   # A100 için AutoBatch
    device=0,
    project=str(RUN_ROOT),
    name=RUN_NM,
    workers=8,
    deterministic=True,
    cache="disk",
    seed=42,

    # ---- OPTIM (yumuşak FT) ----
    optimizer="AdamW",
    lr0=1.2e-3, lrf=1e-2, warmup_epochs=5, cos_lr=True,
    momentum=0.9, weight_decay=0.05,
    freeze=0,                   # katman dondurmak istersen örn. freeze=10

    # ---- Kayıp ağırlıkları: kutu kalitesi/konumu öncelikli ----
    box=8.0, cls=0.35, dfl=1.5,

    # ---- AUG (renk bağımlılığını kır + şekli koru) ----
    #  - hsv_* ile geniş renk jitter
    #  - küçük geometri değişimleri (kenar/şekil öğrenmesini koru)
    #  - mosaic düşük, mixup/copy_paste kapalı (yanlış bağlamsal ipuçlarını azalt)
    hsv_h=0.020, hsv_s=0.60, hsv_v=0.45,
    degrees=8.0, shear=2.0, translate=0.05, scale=0.20, perspective=0.0005,
    fliplr=0.5, flipud=0.0,
    mosaic=0.20, mixup=0.0, copy_paste=0.0,
    close_mosaic=10,            # son 10 epok mozaik kapansın
    rect=True,                  # ince/uzun objede daha tutarlı kutular

    # ---- Erken durdurma (60 epok hedefiyle) ----
    patience=15,                # 10–20 aralığı idealdir; burada 15
    verbose=True
)

print("\n[OK] FT run:", RUN_ROOT / RUN_NM)
print("Weights:", RUN_ROOT / RUN_NM / "weights")

New https://pypi.org/project/ultralytics/8.3.202 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=8.0, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.35, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/censorly-training/violence/data_violence_v11_ft_20250919_1419.yaml, degrees=8.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.02, hsv_s=0.6, hsv_v=0.45, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0012, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/censorly

In [ ]:
from ultralytics.utils.metrics import ConfusionMatrix

BEST = RUN_ROOT / RUN_NM / "weights" / "best.pt"
m = YOLO(str(BEST))
print("Loaded:", BEST)

def eval_split(split, conf=0.40, iou=0.65):
    r = m.val(data=str(yaml_path), split=split, imgsz=IMGSZ, conf=conf, iou=iou, device=0, plots=False, verbose=False)
    return float(r.box.map50), float(r.box.p), float(r.box.r), float(r.box.f1)

# Hızlı süpürme: FP’yi azaltırken R’yi çok düşürmeyen eşiği seç
cands_conf = np.linspace(0.30, 0.70, 9)   # 0.30–0.70 arası
cands_iou  = [0.55, 0.60, 0.65]           # NMS IoU

best = None
print("\n[SWEEP] split=val")
for iou in cands_iou:
    for conf in cands_conf:
        mAP50, P, R, F1 = eval_split("val", conf, iou)
        score = 0.60*P + 0.40*R          # FP duyarlılığı yüksek ise P’yi daha çok tart
        row   = (score, conf, iou, P, R, F1, mAP50)
        if (best is None) or (row[0] > best[0]): best = row
        print(f"conf={conf:.2f} iou={iou:.2f} | P={P:.3f} R={R:.3f} F1={F1:.3f} mAP50={mAP50:.3f} score={score:.3f}")

best_score, CONF_THR, IOU_THR, P, R, F1, mAP50 = best
print(f"\n[SELECTED] conf={CONF_THR:.2f}  iou={IOU_THR:.2f}  | P={P:.3f} R={R:.3f} F1={F1:.3f} mAP50={mAP50:.3f}")

# Seçilen eşiklerle test
mAP50_t, P_t, R_t, F1_t = eval_split("test", CONF_THR, IOU_THR)
print(f"[TEST] conf={CONF_THR:.2f} iou={IOU_THR:.2f} | P={P_t:.3f} R={R_t:.3f} F1={F1_t:.3f} mAP50={mAP50_t:.3f}")

Loaded: /content/drive/MyDrive/censorly-training/violence/yolov8m_ft_v2_shapeBiasFix_960_e60_20250919_1454/weights/best.pt

[SWEEP] split=val
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 0.8±1.5 MB/s, size: 1005.8 KB)
val: Scanning /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/labels/test.cache... 602 images, 196 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 636/636 793.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 40/40 3.7it/s 10.8s
                   all        636        645      0.787      0.549      0.684      0.479
Speed: 0.2ms preprocess, 4.7ms inference, 0.0ms loss, 0.9ms postprocess per image
conf=0.30 iou=0.55 | P=0.787 R=0.549 F1=0.647 mAP50=0.684 score=0.692
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+

/tmp/ipython-input-1058092749.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(r.box.map50), float(r.box.p), float(r.box.r), float(r.box.f1)


val: Fast image access ✅ (ping: 0.6±0.1 ms, read: 2.4±5.1 MB/s, size: 85.1 KB)
val: Scanning /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/labels/test.cache... 602 images, 196 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 636/636 850.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 40/40 3.1it/s 12.7s
                   all        636        645      0.787      0.549      0.685      0.483
Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 0.7ms postprocess per image
conf=0.35 iou=0.55 | P=0.787 R=0.549 F1=0.647 mAP50=0.685 score=0.692
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.5±0.1 ms, read: 24.2±12.2 MB/s, size: 62.5 KB)
val: Scanning /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/labels/test.cache... 602 images, 196 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 636/636 852.0Kit/s 0.0s
             

Exception in thread Thread-42 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 61, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/resource_

conf=0.40 iou=0.55 | P=0.775 R=0.557 F1=0.648 mAP50=0.691 score=0.688
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 6.5±13.5 ms, read: 12.9±17.5 MB/s, size: 711.5 KB)
val: Scanning /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/labels/test.cache... 602 images, 196 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 636/636 760.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 40/40 4.5it/s 8.8s
                   all        636        645      0.794      0.543       0.69      0.493
Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 0.8ms postprocess per image
conf=0.45 iou=0.55 | P=0.794 R=0.543 F1=0.645 mAP50=0.690 score=0.693
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 112.6±158.1 MB/s, size: 282.5 KB)
val: Scanning /content

Exception in thread Thread-49 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 61, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/resource_

conf=0.65 iou=0.55 | P=0.893 R=0.453 F1=0.601 mAP50=0.676 score=0.717
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.4±0.2 ms, read: 30.8±29.3 MB/s, size: 76.5 KB)
val: Scanning /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/labels/test.cache... 602 images, 196 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 636/636 877.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 40/40 4.8it/s 8.4s
                   all        636        645       0.92      0.409      0.664      0.502
Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 0.6ms postprocess per image
conf=0.70 iou=0.55 | P=0.920 R=0.409 F1=0.567 mAP50=0.664 score=0.716
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 2.9±4.4 MB/s, size: 840.5 KB)
val: Scanning /content/drive

In [ ]:
model = YOLO(str(BASE_BEST))  # önceki FT modelinden DEVAM

res = model.train(
    data=str(yaml_path),
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=-1,                   # A100 için AutoBatch
    device=0,
    project=str(RUN_ROOT),
    name=RUN_NM,
    workers=8,
    deterministic=True,
    cache="disk",
    seed=42,

    # ---- OPTIM (yumuşak FT) ----
    optimizer="AdamW",
    lr0=1.2e-3, lrf=1e-2, warmup_epochs=5, cos_lr=True,
    momentum=0.9, weight_decay=0.05,
    freeze=0,                   # katman dondurmak istersen örn. freeze=10

    # ---- Kayıp ağırlıkları: kutu kalitesi/konumu öncelikli ----
    box=8.0, cls=0.35, dfl=1.5,

    # ---- AUG (hatalara göre düzenlendi) ----
    # Renk yanlılığını kır: daha geniş satürasyon + orta parlaklık jitter
    hsv_h=0.020,
    hsv_s=0.65,   # ↑ (metal rengine aşırı bağlanmayı azaltır)
    hsv_v=0.50,

    # Yön/poz varyasyonu: dik ve açılı silahlar
    degrees=22.0,       # ±22° yeterli çeşitlilik
    shear=3.0,
    flipud=0.30,        # ↑ ters/dik tutulan silahlar için
    fliplr=0.5,

    # Küçük/uzak ve kısmi örtülere dayanıklılık
    translate=0.07,     # hafif kaydırma
    scale=0.30,         # formu çok bozmadan zoom in/out
    perspective=0.0008, # çok küçük tut (kenarları aşırı bozmasın)

    # Kısmi örtülme ve “parlak metal ipucu” bağımlılığını azaltma
    erasing=0.55,       # CoarseDropout olasılığı (biraz agresif ama dengeli)

    # Bağlam çeşitliliği ama formu koru
    mosaic=0.30,        # önceki 0.20’den biraz daha fazla (küçük objeye yardımcı)
    mixup=0.05,         # çok küçük, sahneyi büsbütün bozmaz
    copy_paste=0.0,
    close_mosaic=15,    # son 15 epok mozaik kapansın (stabilizasyon)

    # İnce/uzun kutular için
    rect=True,
    # ---- Erken durdurma (60 epok hedefiyle) ----
    patience=15,                # 10–20 aralığı idealdir; burada 15
    verbose=True
)

print("\n[OK] FT run:", RUN_ROOT / RUN_NM)
print("Weights:", RUN_ROOT / RUN_NM / "weights")

New https://pypi.org/project/ultralytics/8.3.202 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.200 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=8.0, cache=disk, cfg=None, classes=None, close_mosaic=15, cls=0.35, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/censorly-training/violence/data_violence_v11_ft_20250918_1604.yaml, degrees=22.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.55, exist_ok=False, fliplr=0.5, flipud=0.3, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.02, hsv_s=0.65, hsv_v=0.5, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0012, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=/content/drive/MyDrive/censo

In [ ]:
yaml_text = f"""
# Gun/Knife two-class dataset
path: .
train:
  - {V2_IMG/'train'}
  - {F_IMG/'train'}
val:
  - {V2_IMG/'validation'}
  - {F_IMG/'validation'}
test:
  - {V2_IMG/'test'}
  - {F_IMG/'test'}
nc: 2
names: [Gun, Knife]
"""
yaml_path = Path('/content/violence_gun_knife.yaml')
yaml_path.write_text(yaml_text)
print(yaml_path.read_text())



# Gun/Knife two-class dataset
path: .
train:
  - /content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence-2/images/train
  - /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/train
val:
  - /content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence-2/images/validation
  - /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/validation
test:
  - /content/drive/MyDrive/censorly-dataset/Violence/Violence-True/Violence-2/images/test
  - /content/drive/MyDrive/censorly-dataset/Violence/Violence-False/images/test
nc: 2
names: [Gun, Knife]



In [ ]:
# Knife görüntülerini tespit etmek için basit sezgisel: label satırında "1 " geçen dosyalar
knife_stems = set()
for sp in splits:
    for lbl in (V2_LBL/sp).rglob('*.txt'):
        txt = lbl.read_text()
        if '\n1 ' in ('\n'+txt) or txt.startswith('1 '):
            knife_stems.add(lbl.stem)

train_list = []
for p in sorted((V2_IMG/'train').rglob('*.*')):
    if p.suffix.lower() in {'.jpg','.jpeg','.png','.bmp'}:
        times = 2 if p.stem in knife_stems else 1
        train_list.extend([str(p)]*times)

# Negatifleri 1x ekle
for p in sorted((F_IMG/'train').rglob('*.*')):
    if p.suffix.lower() in {'.jpg','.jpeg','.png','.bmp'}:
        train_list.append(str(p))

list_path = Path('/content/train_oversampled.txt')
list_path.write_text('\n'.join(train_list) + '\n')
print('Oversampled train list size:', len(train_list))

yaml_text2 = f"""
path: .
train: {list_path}
val:
  - {V2_IMG/'validation'}
  - {F_IMG/'validation'}
test:
  - {V2_IMG/'test'}
  - {F_IMG/'test'}
nc: 2
names: [Gun, Knife]
"""
yaml_path = Path('/content/violence_gun_knife.yaml')
yaml_path.write_text(yaml_text2)
print('YAML updated to use oversampled train list.')


Oversampled train list size: 1967
YAML updated to use oversampled train list.


In [ ]:
from ultralytics import YOLO

# A) Tek-sınıf modelinden devam:
START_FROM = '/content/drive/MyDrive/best_models/violence_best.pt'

# # B) Tarafsız başlangıç:
# START_FROM = 'yolo11m.pt'   # veya 'yolo11s.pt' (daha hafif)

model = YOLO(START_FROM)

results = model.train(
    data=str(yaml_path),
    epochs=80, imgsz=1280, batch=12,            # Knife küçük obje → yüksek çözünürlük iyi
    lr0=0.004, lrf=0.2, momentum=0.9, weight_decay=5e-4,
    mosaic=0.3, mixup=0.05,                     # mosaic düşük: ince silüet bozulmasın
    degrees=4, scale=0.12, translate=0.05, shear=1.0,
    hsv_h=0.015, hsv_s=0.5, hsv_v=0.3,
    copy_paste=0.25,
    patience=30, pretrained=True,
    project='/content/drive/MyDrive/runs-violence',
    name='gun_knife_v1', exist_ok=True
)

best = results.save_dir / 'weights' / 'best.pt'
print('Best:', best, best.exists())


Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.25, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/violence_gun_knife.yaml, degrees=4, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.3, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.004, lrf=0.2, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=/content/drive/MyDrive/best_models/violence_best.pt, momentum=0.9, mosaic=0.3, multi_scale=False, name=gun_knife_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overl

ImportError: cannot import name '_center' from 'numpy._core.umath' (/usr/local/lib/python3.12/dist-packages/numpy/_core/umath.py)

In [ ]:
# === Build a balanced train list (1:1 ±) ===
from pathlib import Path
import random
BASE = Path('/content/drive/MyDrive/censorly-dataset/Violence')
P_IMG = BASE/'Violence-True'/'Violence-2'/'images'/'train'
N_IMG = BASE/'Violence-False'/'images'/'train'


def imgs(d):
    return [p for p in d.rglob('*') if p.suffix.lower() in {'.jpg','.jpeg','.png','.bmp'}]
pos = imgs(P_IMG); neg = imgs(N_IMG)

# Knife hafif oversample (2x)
LBL = BASE/'Violence-True'/'Violence-2'/'labels'/'train'
knife_stems = {p.stem for p in LBL.rglob('*.txt') if p.read_text().strip().startswith('1 ') or '\n1 ' in ('\n'+p.read_text())}
pos_os = []
for p in pos:
    pos_os.append(p)
    if p.stem in knife_stems:
        pos_os.append(p)  # +1 kopya

# Negatifleri kapa (1:1.1 hedef)
import random
random.shuffle(pos_os); random.shuffle(neg)
target_neg = min(len(neg), int(len(pos_os)*1.1))
neg_sel = neg[:target_neg]

train_list = [str(p) for p in pos_os+neg_sel]
Path('/content/train_balanced.txt').write_text('\n'.join(train_list)+'\n')
print('pos_os:', len(pos_os), 'neg_sel:', len(neg_sel))


pos_os: 1581 neg_sel: 386


In [ ]:
yaml = f"""
path: .
train: /content/train_balanced.txt
val:
  - {BASE/'Violence-True'/'Violence-2'/'images'/'validation'}
  - {BASE/'Violence-False'/'images'/'validation'}
test:
  - {BASE/'Violence-True'/'Violence-2'/'images'/'test'}
  - {BASE/'Violence-False'/'images'/'test'}
nc: 2
names: [Gun, Knife]
"""
from pathlib import Path
Path('/content/gun_knife_bal.yaml').write_text(yaml)


433

In [ ]:
from ultralytics import YOLO

MODEL = '/content/drive/MyDrive/best_models/violence_best.pt'  # mevcut en iyi ağırlığın
YAML  = '/content/gun_knife_bal.yaml'

m = YOLO(MODEL)

# Aşama-1: backbone freeze (ısınma, agresif değil)
m.train(
  data=YAML, epochs=8, freeze=10, imgsz=1280, batch=12,
  lr0=0.003, lrf=0.2, momentum=0.9, weight_decay=5e-4,
  mosaic=0.25, mixup=0.0, degrees=3, scale=0.10, translate=0.03, shear=0.5,
  hsv_h=0.012, hsv_s=0.4, hsv_v=0.25,
  patience=8, project='/content/drive/MyDrive/runs-violence', name='gk_bal_warmup', exist_ok=True
)

# Aşama-2: unfreeze + küçük LR (denge odaklı)
WARM = '/content/drive/MyDrive/runs-violence/gk_bal_warmup/weights/best.pt'
m2 = YOLO(WARM)
m2.train(
  data=YAML, epochs=42, imgsz=1280, batch=12,
  lr0=0.002, lrf=0.2, momentum=0.9, weight_decay=5e-4,
  mosaic=0.25, mixup=0.0, degrees=3, scale=0.10, translate=0.03, shear=0.5,
  hsv_h=0.012, hsv_s=0.4, hsv_v=0.25,
  patience=15, project='/content/drive/MyDrive/runs-violence', name='gk_bal_main', exist_ok=True
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gun_knife_bal.yaml, degrees=3, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=8, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.012, hsv_s=0.4, hsv_v=0.25, imgsz=1280, int8=False, iou=0.7,

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a60a61bc140>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804